Hugging face

https://huggingface.co/docs/api-inference/detailed_parameters

In [29]:
import requests
import pandas as pd
import os

In [30]:
API_TOKEN = open('Token.txt').read().strip()

Zero-Shot Classification task

Refunds

In [32]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-mnli"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query(
    {
        "inputs": "Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
        "parameters": {"candidate_labels": ["refund", "legal", "faq"]},
    }
)

In [33]:
data

{'sequence': 'Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!',
 'labels': ['refund', 'faq', 'legal'],
 'scores': [0.8777878284454346, 0.10522636026144028, 0.01698581501841545]}

In [34]:
type(data)

dict

In [35]:
def get_highest_score_label(data):
    max_index = data['scores'].index(max(data['scores']))
    highest_label = data['labels'][max_index]
    return f"This sentence refers to {highest_label}"

get_highest_score_label(data)

'This sentence refers to refund'

Likeness

In [36]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-mnli"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query(
    {
        "inputs": "I love this product",
        "parameters": {"candidate_labels": ["good", "intermediate", "bad"]},
    }
)

In [37]:
def get_highest_score_label(data):
    max_index = data['scores'].index(max(data['scores']))
    highest_label = data['labels'][max_index]
    return f"This opinion is {highest_label}"

get_highest_score_label(data)

'This opinion is good'

Multiple inputs

In [38]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-mnli"

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()


def query_multiple_inputs(inputs_list, candidate_labels):
    results = []
    
    for input_text in inputs_list:
        payload = {
            "inputs": input_text,
            "parameters": {"candidate_labels": candidate_labels},
        }
        result = query(payload)
        results.append(result)
    
    return results

In [39]:
inputs = ["I love this product", "This is not what I expected", "It works fine, but could be better"]
candidate_labels = ["good", "neutral", "bad"]

In [40]:
data = query_multiple_inputs(inputs, candidate_labels)

data

[{'sequence': 'I love this product',
  'labels': ['good', 'neutral', 'bad'],
  'scores': [0.9883801341056824, 0.01027941145002842, 0.001340384827926755]},
 {'sequence': 'This is not what I expected',
  'labels': ['bad', 'good', 'neutral'],
  'scores': [0.9213561415672302, 0.04466470703482628, 0.033979207277297974]},
 {'sequence': 'It works fine, but could be better',
  'labels': ['good', 'neutral', 'bad'],
  'scores': [0.927952229976654, 0.06524170935153961, 0.006806021090596914]}]

In [41]:
def process_dict_to_dataframe(data_dict):
    sequences = []
    top_labels = []
    top_scores = []
    for entry in data_dict:
        sequences.append(entry['sequence'])
        max_index = entry['scores'].index(max(entry['scores']))
        top_labels.append(entry['labels'][max_index])
        top_scores.append(entry['scores'][max_index])
    df = pd.DataFrame({
        'Frase': sequences,
        'Etiqueta con Mayor Puntuación': top_labels,
        'Puntuación': top_scores
    })
    
    return df

In [42]:
df = process_dict_to_dataframe(data)
df

,Frase,Etiqueta con Mayor Puntuación,Puntuación
0,I love this product,good,0.988380
1,This is not what I expected,bad,0.921356
2,"It works fine, but could be better",good,0.927952


Text classification

In [43]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query({"inputs": "I like you. I love you"})
data

[[{'label': 'POSITIVE', 'score': 0.9998738765716553},
  {'label': 'NEGATIVE', 'score': 0.0001261125726159662}]]

In [44]:
#MULTIPLE

headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()


def query_multiple_inputs_class (inputs_list):
    results = []
    
    for input_text in inputs_list:
        payload = {
            "inputs": input_text,
        }
        result = query(payload)
        results.append(result)
    
    return results

In [45]:
inputs = ["I love this product", "This is not what I expected", "It works fine, but could be better"]

data = query_multiple_inputs_class(inputs)

In [46]:
data

[[[{'label': 'POSITIVE', 'score': 0.9998788833618164},
   {'label': 'NEGATIVE', 'score': 0.00012105619680369273}]],
 [[{'label': 'NEGATIVE', 'score': 0.9997480511665344},
   {'label': 'POSITIVE', 'score': 0.0002519444387871772}]],
 [[{'label': 'POSITIVE', 'score': 0.9944763779640198},
   {'label': 'NEGATIVE', 'score': 0.005523692816495895}]]]

Image recognition

In [49]:
import json
import requests
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/google/vit-base-patch16-224"
def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))
data = query("cats.jpeg")
data

[{'label': 'Egyptian cat', 'score': 0.43790608644485474},
 {'label': 'tabby, tabby cat', 'score': 0.37949836254119873},
 {'label': 'tiger cat', 'score': 0.1658201366662979},
 {'label': 'lynx, catamount', 'score': 0.00667042238637805},
 {'label': 'Persian cat', 'score': 0.0007664052536711097}]

In [50]:
possible_dog = query("dog.jpeg")
possible_dog

[{'label': 'German shepherd, German shepherd dog, German police dog, alsatian',
  'score': 0.9879292249679565},
 {'label': 'kelpie', 'score': 0.0025923456996679306},
 {'label': 'muzzle', 'score': 0.0010812951950356364},
 {'label': 'malinois', 'score': 0.0010680905543267727},
 {'label': 'Norwegian elkhound, elkhound', 'score': 0.0009105136850848794}]

In [ ]:
house = query("house.jpg")
house

[{'label': 'patio, terrace', 'score': 0.26827412843704224},
 {'label': 'palace', 'score': 0.04998278245329857},
 {'label': 'boathouse', 'score': 0.049633320420980453},
 {'label': 'tile roof', 'score': 0.04291205108165741},
 {'label': 'mobile home, manufactured home', 'score': 0.03886718675494194}]

In [ ]:
house1 = query("house.jpg")
house1

[{'label': 'patio, terrace', 'score': 0.26827412843704224},
 {'label': 'palace', 'score': 0.04998278245329857},
 {'label': 'boathouse', 'score': 0.049633320420980453},
 {'label': 'tile roof', 'score': 0.04291205108165741},
 {'label': 'mobile home, manufactured home', 'score': 0.03886718675494194}]

In [ ]:
trump = query("trump.jpg")
trump

[{'label': 'Windsor tie', 'score': 0.12037745863199234},
 {'label': 'suit, suit of clothes', 'score': 0.08712882548570633},
 {'label': "academic gown, academic robe, judge's robe",
  'score': 0.01740042306482792},
 {'label': 'flagpole, flagstaff', 'score': 0.01593775860965252},
 {'label': 'bearskin, busby, shako', 'score': 0.0115365507081151}]